In [1]:
import pandas as pd
import urllib3 as urllib
import json
import IPython.display

In [2]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [3]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [4]:
#Saving a copy of source data 
ltc_data_json = json.dumps(ltc_data)

In [30]:
deaths = ltc_data['LTC_Reported_Cases']['deaths']
confirmed_cases = ltc_data['LTC_Reported_Cases']['confirmed_cases']

In [5]:
#Saving a copy of source data 
ltc_data_json = json.dumps(ltc_data)

f = open("Source_data/IL_" + reporting_date + "LTC_data_Source.json","w")
f.write(ltc_data_json)
f.close()

In [25]:
df = pd.DataFrame(ltc_data['FacilityValues'])
#ltc_data['LTC_Reported_Cases']
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']

In [26]:
df.to_csv('Reporting_data/IL_' + reporting_date + '_LTC_data_.csv', index = False)

In [31]:
df['num of facilities'] = 1 
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % confirmed_cases)
print ('Deaths:  %d'% deaths)
print ('Facilities: %d' % df.reporting_date.value_counts()[0])

Date: 2020-11-06
Cases: 36683
Deaths:  5253
Facilities: 1309


In [9]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = (df_county['deaths'] / df_county['confirmed_cases'])
df_county.sort_values('deaths', ascending=False).to_csv('Reporting_data/IL_county_stats_' + reporting_date + '.csv')
df_county.sort_values('deaths', ascending=False)

,confirmed_cases,deaths,% deaths,num of facilities
County,,,,
Cook,15294,2450,0.160194,409
DuPage,2872,463,0.161212,118
Lake,2319,342,0.147477,90
Will,1533,212,0.138291,59
Kane,1497,198,0.132265,77
...,...,...,...,...
Effingham,9,0,0.000000,3
Crawford,2,0,0.000000,1
Fulton,5,0,0.000000,1


In [10]:
import altair as alt
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O', sort='-x'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2

alt.HConcatChart(...)

In [23]:
import altair as alt
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2 

alt.HConcatChart(...)

In [ ]:
df.sort_values(by=['deaths'], ascending=False).to_csv('IL_LTC_Covid.csv')

In [12]:
df.reporting_date.value_counts()[0]

1309

In [13]:
df.head()

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths,num of facilities
0,Adams,Chaddock,3,0,Closed,2020-11-06,0.000000,1
1,Adams,Illinois Veterans Home,45,2,Open,2020-11-06,0.044444,1
2,Adams,Senior Center,4,2,Closed,2020-11-06,0.500000,1
3,Adams,Good Samaritan Home,13,1,Open,2020-11-06,0.076923,1
4,Adams,Adams Pointe Senior Living,4,0,Closed,2020-11-06,0.000000,1
